In [1]:
import pandas as pd
from bs4 import BeautifulSoup


In [7]:
questions = pd.read_csv("Questions.csv", encoding='latin1', nrows=5000)
answers = pd.read_csv("Answers.csv", encoding='latin1', nrows=5000)
tags = pd.read_csv("Tags.csv", encoding='latin1', nrows=5000)


In [8]:
def clean_html(raw_html):
    return BeautifulSoup(str(raw_html), "html.parser").get_text()

questions['Body'] = questions['Body'].apply(clean_html)
questions['Title'] = questions['Title'].apply(clean_html)
answers['Body'] = answers['Body'].apply(clean_html)


C:\Users\NTC\AppData\Local\Temp\ipykernel_30376\1027296034.py:2: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  return BeautifulSoup(str(raw_html), "html.parser").get_text()


In [9]:
questions.dropna(subset=['Body', 'Title'], inplace=True)
answers.dropna(subset=['Body'], inplace=True)


In [10]:
answers = answers[answers['Score'] > 5]
questions = questions[questions['Score'] > 5]


In [11]:
merged_df = pd.merge(answers, questions, left_on='ParentId', right_on='Id', suffixes=('_Answer', '_Question'))


In [12]:
tags_grouped = tags.groupby('Id')['Tag'].apply(list).reset_index()
merged_df = pd.merge(merged_df, tags_grouped, left_on='Id_Question', right_on='Id', how='left').drop(columns=['Id'])


In [13]:
merged_df = merged_df.sort_values(by='Score_Answer', ascending=False).head(5000)


In [14]:
merged_df.to_csv("cleaned_stackoverflow.csv", index=False)
